In [1]:
#%pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [2]:
#%pip uninstall -y transformers
#%pip install git+https://github.com/huggingface/transformers

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline,AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from peft import get_peft_model, LoraConfig, TaskType,prepare_model_for_kbit_training
import torch

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import json
import re

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [4]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [5]:
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
print(tokenizer.special_tokens_map)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'bos_token': '<s>', 'eos_token': '<|endoftext|>', 'unk_token': '<unk>', 'pad_token': '<|endoftext|>'}


In [6]:
# 데이터 포인트 예시
data_point = {
    "context": "육신이 흐느적흐느적하도록 피로했을 때만 정신이 은화처럼 맑소",
    "label": "육신이 흐느적흐느적하도록 피로했을 때만 정신이 물처럼 맑소"
}

# 프롬프트 생성 함수
def generate_prompt(data_point):
    return f"""아래 문장의 비유표현을 변환하여 새로운 표현으로 다시 써 주세요..

    ### Instruction:
    {data_point["context"]}

    ### Response:
    {data_point["label"]}"""

# 프롬프트 생성
prompt = generate_prompt(data_point)

# 입력 문장과 목표 문장 토큰화
input_ids = tokenizer(prompt, return_tensors='pt').input_ids
labels = tokenizer(data_point["label"], return_tensors='pt').input_ids

# input_ids와 labels 출력
print(f"Input IDs: {input_ids}")
print(f"Labels: {labels}")

# 토큰을 실제 텍스트로 변환하여 확인
print(f"Decoded Input IDs: {tokenizer.decode(input_ids[0], skip_special_tokens=True)}")
print(f"Decoded Labels: {tokenizer.decode(labels[0], skip_special_tokens=True)}")

Input IDs: tensor([[    1, 29871, 30860,   238,   161,   155, 29871, 31406, 31299, 30708,
         29871, 31487, 31533,   240,   148,   159, 31680, 31286, 29871,   238,
           182,   131,   240,   156,   155, 30944, 31457, 29871,   239,   134,
           139, 30906,   239,   157,   183, 29871,   240,   148,   159, 31680,
           239,   159,   191, 30906, 29871, 30709, 30889, 29871,   239,   144,
           171, 29871, 30981, 31578, 31527,   636,    13,    13,  1678,   835,
          2799,  4080, 29901,    13,   268,   239,   159,   164, 31262, 30393,
         29871,   240,   160,   147,   238,   141,   147,   239,   163,   132,
           240,   160,   147,   238,   141,   147,   239,   163,   132, 30944,
         31136,   238,   164,   160, 29871,   240,   151,   191, 30906,   240,
           153,   139, 31286, 29871,   238,   152,   143, 31826, 29871, 30852,
         31262, 30393, 29871, 31354, 31225,   239,   181,   155,   238,   162,
           191, 29871,   238,   170,   14

In [7]:
with open("./data_label_test_bert/Similar_data_klue_large_KoELECTRA.json","r",encoding="utf-8") as f:
    data = json.load(f)


Similar_raw = []

for item in data["data"]:
    for para in item["para"]:
        context = para["context"]
        for labels in para["label"]:
            label = labels["sequence"]
            token_str = labels["token_str"]
            Similar_raw.append((context,label,token_str))


In [8]:
train_data_raw, test_data_raw = train_test_split(Similar_raw, test_size=0.1)

train_data = pd.DataFrame(train_data_raw)
test_data = pd.DataFrame(test_data_raw)

train_data.rename(columns={0: 'context', 1: 'label', 2: 'token'}, inplace=True)
test_data.rename(columns={0: 'context', 1: 'label', 2: 'token'}, inplace=True)
train_data = train_data.drop('token', axis=1)
train_data.reset_index(inplace=True)
test_data.reset_index(inplace=True)
train_data.head(5)

,index,context,label
0,0,알듯 한곳도 모르는 대목도 많은 것을 이를 악물고 시험공부하듯이 대들었으나 날이 거...,알듯 한곳도 모르는 대목도 많은 것을 이를 악물고 시험공부하듯이 대들었으나 날이 거...
1,1,동으로 동으로 목말라 찾던 어머니인 땅이 인제 살 바치는 성찬은 이뿐이던가 저주받을...,동으로 동으로 목말라 찾던 어머니인 땅 하늘 인제 살 바치는 성찬은 하늘 뿐 하늘...
2,2,옷을 갈아입은 아내는 딴사람처럼 어여삐 보인다.,옷을 갈아입은 아내는 딴 여자 처럼 어여삐 보인다.
3,3,의복으로 말한대도 양복바지가 조선에 건너오자 이 본을 떠서 한동안은 홀태바지에 홀태...,의복으로 말한대도 양복바지가 조선에 건너오자 이 본을 떠서 한동안은 홀태바지에 홀태...
4,4,바다 실어다 뿌리는 바람처럼 씌워 타.,바다 실어다 뿌리는 것 처럼 씌워 타.


In [9]:
BOS = tokenizer.bos_token
EOS = tokenizer.eos_token
PAD = tokenizer.pad_token

In [10]:
def generate_prompt(data_point):
    return f"""아래 문장의 비유표현을 변환하여 새로운 표현으로 다시 써 주세요..

    ### Instruction:
    {data_point["context"]}

    ### Response:
    {data_point["label"]}"""

In [11]:
#데이터 처리 클래스
class Similar_Dataset(Dataset):
    def __init__(self, chats, max_len=512):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.bos= BOS
        self.eos = EOS
        self.pad = PAD
        self.tokenizer = tokenizer
    def __len__(self):  
        return len(self._data)

    def __getitem__(self, idx):  # 로드한데이터를 차례차례 DataLoader로 넘김
        data_point = self._data.iloc[idx]
        prompt = generate_prompt(data_point)
        l = data_point["label"]  # label을 가져온다.

        prompt_toked = self.tokenizer.tokenize(self.bos + prompt + self.eos)
        prompt_len = len(prompt_toked)
        
        l_toked = self.tokenizer.tokenize(self.bos + l + self.eos)
        l_len = len(l_toked)

        #최대길이 초과시 처리
        if prompt_len> self.max_len:
            prompt_len = self.max_len
            prompt_toked = prompt_toked[:prompt_len]
       
        if l_len> self.max_len:
            l_len = self.max_len
            l_toked = l_toked[:l_len]

        labels_ids = self.tokenizer.convert_tokens_to_ids(l_toked)        
       
        while len(labels_ids) < self.max_len: # 최대길이만큼 PADDING
            labels_ids += [self.tokenizer.pad_token_id]

        token_ids = self.tokenizer.convert_tokens_to_ids(prompt_toked)
      
        while len(token_ids) < self.max_len: # 최대길이만큼 PADDING
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, labels_ids)

In [12]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    label = [item[1] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(label)

In [13]:
train_set = Similar_Dataset(train_data, max_len=512)
loader = DataLoader(train_set, batch_size=4,  num_workers=0, shuffle=True, collate_fn=collate_batch,)
len(loader)

4435

In [14]:
for i, batch in enumerate(loader):
    token_ids, labels_ids = batch
    print(f"Batch {i+1}")
    print("Token IDs:", token_ids)
    print(len(token_ids[0]))
    print(token_ids.shape)
    print(token_ids[0][7])
    print(tokenizer.decode(token_ids[0][7]))
    print(tokenizer.decode(token_ids[0]))

    print(len(token_ids))
    print("Labels IDs:", labels_ids)
    print("Labels IDs:", (tokenizer.decode(labels_ids[0])))
    break

Batch 1
Token IDs: tensor([[    1, 29871, 30860,  ..., 32000, 32000, 32000],
        [    1, 29871, 30860,  ..., 32000, 32000, 32000],
        [    1, 29871, 30860,  ..., 32000, 32000, 32000],
        [    1, 29871, 30860,  ..., 32000, 32000, 32000]])
512
torch.Size([4, 512])
tensor(31406)
문
<s> 아래 문장의 비유표현을 변환하여 새로운 표현으로 다시 써 주세요..

    ### Instruction:
    남들이 이야기하는 것처럼 저어 하늘 위의 천당에 가 계신지……, 언니는 나처럼 맞지나 않고 잘 지내는지……, 우리 언니도 나처럼 날마다 얻어맞기나 하면 어쩔까?

    ### Response:
     남들이 이야기하는 것처럼 저어 하늘 위의 천당에 가 계신지 … …, 언니는 남편 처럼 맞지 남편 않고 잘 지내는지 … …, 우리 언니도 남편 처럼 날마다 얻어맞기 남편 하면 어쩔까? <|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endof

In [15]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"]
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = get_peft_model(model, lora_config)
model.to(device)
model.train()

cuda


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3Attention(
              (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
              (qkv_proj): Linear(
                in_features=3072, out_features=9216, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=9216, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): 

In [16]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-5)
# 스케쥴러 설정
epochs = 10
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(loader)*epochs)

# 학습 루프
for epoch in range(epochs):
    total_loss = 0
    progress_bar = tqdm(loader, desc="Epoch {:1d}".format(epoch+1), leave=False, disable=False)
    for batch in progress_bar:
        batch = tuple(t.to(device) for t in batch)
        input_ids, labels = batch

        # 모델 forward 및 loss 계산
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss

        # 백워드 및 옵티마이저 스텝
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item() / len(batch))})

    print("Epoch {}/{} - Loss: {:.4f}".format(epoch+1, epochs, total_loss / len(loader)))

/home/work/.local/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1:   0%|          | 0/4435 [00:00<?, ?it/s]You are not running the flash-attention implementation, expect numerical differences.


Epoch 1/10 - Loss: 1.3965


Epoch 2/10 - Loss: 1.2413


Epoch 3/10 - Loss: 1.1948


Epoch 4/10 - Loss: 1.1596


Epoch 5:  10%|▉         | 435/4435 [02:30<23:01,  2.89it/s, training_loss=0.541]

In [ ]:
model.save_pretrained("./phi_3_similar_last")
tokenizer.save_pretrained("./phi_3_similar_last")


3942